# Utility code for identifying primes and prime covered by $\mathcal{G}(29^\#)$
This is very specific code for building arrays of gaps between primes covered by the
cycle $\mathcal{G}(29^\#)$ and beyond

In [1]:
import numpy as np
import array
import gc
import psutil
import sys

In [2]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 5351.89 MB


In [3]:
primes23 = np.load('primes23.npy')
print(f"{len(primes23)} primes from {primes23[0]} to {primes23[-1]}")

12283522 primes from 29 to 223092827


In [4]:
firstprimes = np.array((3,5,7,11,13,17,19,23))
primeslist = np.concatenate((firstprimes,primes23[0:2500000]))
print(f" {len(primeslist)} primes from {primeslist[0]} to {primeslist[-1]}")

 2500008 primes from 3 to 41161949


In [5]:
# just sampling the primes around 3.4E7 : the square is above 1E15
primeslist[2146500:2146550]

array([34995413, 34995449, 34995463, 34995479, 34995487, 34995491,
       34995523, 34995533, 34995553, 34995577, 34995617, 34995659,
       34995661, 34995677, 34995713, 34995749, 34995767, 34995781,
       34995787, 34995797, 34995799, 34995803, 34995817, 34995823,
       34995881, 34995889, 34995899, 34995913, 34995923, 34995943,
       34995959, 34996033, 34996051, 34996057, 34996061, 34996079,
       34996103, 34996121, 34996123, 34996139, 34996141, 34996147,
       34996151, 34996163, 34996187, 34996189, 34996193, 34996201,
       34996211, 34996223])

In [6]:
# identify the primes among odd numbers in a block of size blocklength starting at x0
def pblock(x0, blocklength):
    i=0 # index for primeslist, start at p=3
    j=0 # index for testlength
    p = primeslist[i]
    maxp = np.sqrt(x0+2*blocklength)

    # value associated to index i is xx0+2i
    is_prime = np.ones(blocklength,dtype=bool)
    print(f"x0 {x0} block {len(is_prime)} maxp {maxp}")

    if (x0 <= maxp):  # the starting point is within the range of primes needed in the calculation
        xx0 = maxp
    else:
        xx0 = x0

    if ((xx0 % 2) == 0):  # our starting point xx0 must be odd
        xx0 = xx0+1
    else:
        xx0 = xx0
        
    while (p < maxp):
        print(f"checking {p} of {maxp}", end='\r')
        res0 = xx0 % p  # starting residue
        delta0 = int(p - res0)
        if (res0 == 0):
            j = 0
        elif ((delta0 % 2) ==0):
            j = int(delta0 /2)
        else:
            j = int((p+delta0)/2)
        while (j < blocklength):
            is_prime[j] = False
            j += p
        i += 1
        p = primeslist[i]

    pdexes = np.array(np.where(is_prime))
    pdexes.shape

    newprimes = (2*pdexes[0]+xx0)
    print(f"{len(newprimes)} primes from {newprimes[0]} to {newprimes[-1]}")
    return newprimes

## Example use
We set a large value for a starting point $x_0$ and a large interval over which to search.
The range of our search is bounded by the square of the largest prime we have available in primeslist[].

In [7]:
pfacmax = primeslist[-5]
x0 = pfacmax**2 - 1000  # we want to focus on intervals of survival, so we start at a square and allow for starting in a large gap
x0

np.int64(1694296166624241)

In [8]:
testlength = 900000000

In [9]:
ptrial = pblock(x0,testlength)

x0 1694296166624241 block 900000000 maxp 41161850.86489966
checking 2831329 of 41161850.86489966

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



checking 4123099 of 41161850.86489966

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



checking 6415007 of 41161850.86489966

...

checking 22308863 of 41161850.86489966

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



checking 30968293 of 41161850.86489966

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



51337046 primes from 1694296166624249 to 1694297966624161


In [10]:
ptrial[-10:]

array([1694297966623687, 1694297966623723, 1694297966623753,
       1694297966623831, 1694297966623943, 1694297966623957,
       1694297966623963, 1694297966624099, 1694297966624143,
       1694297966624161])

In [12]:
newpgaps= ptrial[1:len(ptrial)]-ptrial[0:(len(ptrial)-1)]
newpgaps[0:20]

array([ 14,  24,  42,  60,   4,  26,  10,  98,   4,  26,   6,  60,  64,
        72,  42,  38,  12,  48,   4, 206])

In [14]:
# SAVE this data for use in other modules
# we comment out these lines to prevent accidental clobbering of the existing files
#np.save('primeblock111.npy',ptrial)
#np.save('pgapblock111.npy',newpgaps)

In [20]:
# comparing the largest prime found to the tail of primeslist
np.sqrt(ptrial[-1])

np.float64(41161850.86489869)

In [19]:
primeslist[-6:]

array([41161823, 41161829, 41161849, 41161859, 41161943, 41161949])